In [11]:
from juliacall import Main as jl
import numpy as np
# Load Clarabel in Julia
jl.seval('using Clarabel, LinearAlgebra, SparseArrays')
jl.seval('using CUDA, CUDA.CUSPARSE')

# Example: Solve a simple second-order cone programming (SOCP) problem

In [12]:
jl.seval('''
    P = spzeros(3,3)
    q = [0, -1., -1]
    A = SparseMatrixCSC([1. 0 0; -1 0 0; 0 -1 0; 0 0 -1])
    b = [1, 0., 0., 0.]

    # 0-cone dimension 1, one second-order-cone of dimension 3
    cones = Dict("f" => 1, "q"=> [3])

    settings = Clarabel.Settings(direct_solve_method = :cudss)
                                    
    solver   = Clarabel.Solver(P,q,A,b,cones, settings)
    Clarabel.solve!(solver)
    
    # Extract solution
    x = solver.solution
''')



-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 3
  constraints   = 4
  nnz(P)        = 0
  nnz(A)        = 4
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : SecondOrder = 1,  numel = 3

settings:
  linear algebra: direct / cudss, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-12, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       gap     

>>> Clarabel - Results
Status: SOLVED
Iterations: 5
Objective: -1.414
Solve time: 32.2ms


In [13]:
b_new = np.array([2.0, 1.0, 1.0, 1.0], dtype=np.float64)
jl.seval('b_gpu = CuVector{Float64,CUDA.UnifiedMemory}(b)')     #create a vector b_gpu that utilizes unified memory
jl.copyto_b(jl.b_gpu, b_new)                                    #directly copy a cpu vector b_new to a gpu vector b_gpu with unified memory

#############################################
# jl.seval('''
#     Clarabel.update_b!(solver,b)
#     Clarabel.solve!(solver)
# ''')
#############################################

# "_b" is the replacement of "!" in julia function
jl.Clarabel.update_b_b(jl.solver,jl.b_gpu)          #Clarabel.update_b!()
jl.Clarabel.solve_b(jl.solver)                  #Clarabel.solve!()

-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 3
  constraints   = 4
  nnz(P)        = 0
  nnz(A)        = 4
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : SecondOrder = 1,  numel = 3

settings:
  linear algebra: direct / cudss, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-12, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       gap     

>>> Clarabel - Results
Status: SOLVED
Iterations: 5
Objective: -2.243
Solve time: 57.7ms


In [14]:
# Retrieve the solution from Julia to Python
solution = np.array(jl.solver.solution.x)
print("Solution:", solution)

Solution: [2.         1.12132034 1.12132034]


In [15]:
# Example: Solve a simple quadratic programming (QP) problem
jl.seval('''
    P = CuSparseMatrixCSR(spzeros(3,3))
    q = CuVector([0, -1., -1])
    A = CuSparseMatrixCSR(SparseMatrixCSC([1. 0 0; -1 0 0; 0 -1 0; 0 0 -1]))
    b = CuVector([1, 0., 0., 0.])

    # 0-cone dimension 1, one second-order-cone of dimension 3
    # cones = Dict("f" => 1, "q"=> [3])
    cones = [Clarabel.ZeroConeT(1), Clarabel.SecondOrderConeT(3)]

    settings = Clarabel.Settings(direct_solve_method = :cudss)
                                    
    solver   = Clarabel.Solver(P,q,A,b,cones, settings)
    Clarabel.solve!(solver)
    
    # Extract solution
    x = solver.solution
''')

-------------------------------------------------------------
           Clarabel.jl v0.10.0  -  Clever Acronym              
                   (c) Paul Goulart                          
                University of Oxford, 2022                   
-------------------------------------------------------------

problem:
  variables     = 3
  constraints   = 4
  nnz(P)        = 0
  nnz(A)        = 4
  cones (total) = 2
    : Zero        = 1,  numel = 1
    : SecondOrder = 1,  numel = 3

settings:
  linear algebra: direct / cudss, precision: Float64
  max iter = 200, time limit = Inf,  max step = 0.990
  tol_feas = 1.0e-08, tol_gap_abs = 1.0e-08, tol_gap_rel = 1.0e-08,
  static reg : on, ϵ1 = 1.0e-08, ϵ2 = 4.9e-32
  dynamic reg: on, ϵ = 1.0e-13, δ = 2.0e-07
  iter refine: on, reltol = 1.0e-12, abstol = 1.0e-12, 
               max iter = 10, stop ratio = 5.0
  equilibrate: on, min_scale = 1.0e-04, max_scale = 1.0e+04
               max iter = 10

iter    pcost        dcost       gap     

>>> Clarabel - Results
Status: SOLVED
Iterations: 5
Objective: -1.414
Solve time: 54.0ms
